## 图的处理

图用于建模对象（节点）之间的成对关系（边）。在 PyG 中，一个图由 `torch_geometric.data.Data` 的实例描述，默认情况下具有以下属性：

- `data.x`：节点特征矩阵，形状为 `[num_nodes, num_node_features]`
- `data.edge_index`：COO 格式的图连接，形状为 `[2, num_edges]`，类型为 `torch.long`
- `data.edge_attr`：边特征矩阵，形状为 `[num_edges, num_edge_features]`
- `data.y`：训练目标（可能具有任意形状），例如，节点级别目标的形状为 `[num_nodes, *]` 或图级目标的形状为 `[1, *]`
- `data.pos`：节点位置矩阵，形状为 `[num_nodes, num_dimensions]`

这些属性都不是必需的。实际上，`Data` 对象甚至不限于这些属性。我们可以通过 `data.face` 来扩展它，以保存 3D 网格中三角形的连接性，使用形状为 `[3, num_faces]` 的张量和类型为 `torch.long`。

> **注意：**  
> PyTorch 和 `torchvision` 将一个示例定义为图像和目标的元组。我们在 PyG 中省略了这种表示，以便以干净和可理解的方式允许各种数据结构。



In [26]:
from torch_geometric.data import Data

import torch

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
data

Data(x=[3, 1], edge_index=[2, 4])

In [27]:
data.validate(raise_on_error=True)


True

```python
    def forward(self, atomic_numbers: Tensor, atomic_positions: Tensor):
        """
        神经网络前向传播
        :param atomic_numbers: the atomic number of each atom in the molecule, shape: batch_size x atomic_number
        :param atomic_positions:  the coordinate of each atom in the molecule, shape: batch_size x atomic_number x 3
        :return:
         - atomic_number_embedding: Embeddings of atomic numbers with layer normalization applied.
                                           Shape: (batch_size, atomic_number, hid_dim)
         - atomic_adjacency_matrix: Smoothed adjacency matrix representing atomic connections.
                                           Shape: (batch_size, atomic_number, atomic_number)
         - normalized_atom_position_distances: Normalized vectors representing interatomic distances.
                                                      Shape: (batch_size, atomic_number, atomic_number, 3)
         - edge_features: Edge features computed using RBF, representing interatomic relationships.
                                 Shape: (batch_size, atomic_number, atomic_number, ef_dim)
        """
        batch_size, num_atoms, _ = atomic_positions.shape
        # TODO: change the adjacent matrix into the edge list to dramaticlly reduce thesize

        (
            atomic_number_embedding,
            atomic_adjacency_matrix,
            normalized_atom_position_distances,
            edge_features,
        ) = self.mol2graph(atomic_numbers, atomic_positions)
        mask = self.ef_proj(edge_features) * atomic_adjacency_matrix.unsqueeze(-1)

        # TODO: 现在把这个邻接矩阵的表示变成edge list的形式
```

In [28]:
import torch.nn as nn
import torch

In [29]:
batch_size = 32
atomic_number = 96
hidden_channels = 256

In [78]:
# ```python
#     def forward(self, atomic_numbers: Tensor, atomic_positions: Tensor):
#         """
#         神经网络前向传播
#         :param atomic_numbers: the atomic number of each atom in the molecule, shape: batch_size x atomic_number
#         :param atomic_positions:  the coordinate of each atom in the molecule, shape: batch_size x atomic_number x 3
#         :return:
#          - atomic_number_embedding: Embeddings of atomic numbers with layer normalization applied.
#                                            Shape: (batch_size, atomic_number, hid_dim)
#          - atomic_adjacency_matrix: Smoothed adjacency matrix representing atomic connections.
#                                            Shape: (batch_size, atomic_number, atomic_number)
#          - normalized_atom_position_distances: Normalized vectors representing interatomic distances.
#                                                       Shape: (batch_size, atomic_number, atomic_number, 3)
#          - edge_features: Edge features computed using RBF, representing interatomic relationships.
#                                  Shape: (batch_size, atomic_number, atomic_number, ef_dim)
#         """
#         batch_size, num_atoms, _ = atomic_positions.shape
#         # TODO: change the adjacent matrix into the edge list to dramaticlly reduce thesize

#         (
#             atomic_number_embedding,
#             atomic_adjacency_matrix,
#             normalized_atom_position_distances,
#             edge_features,
#         ) = self.mol2graph(atomic_numbers, atomic_positions)
#         mask = self.ef_proj(edge_features) * atomic_adjacency_matrix.unsqueeze(-1)

#         # TODO: 现在把这个邻接矩阵的表示变成edge list的形式
# ```
atomic_embedding = torch.randn(batch_size, atomic_number, hidden_channels)
# Create a random lower triangular matrix
atomic_adjacency_matrix = torch.randint(low=0, high=2, size=(batch_size, atomic_number,  atomic_number))




normalized_atom_position_distances = torch.randn(batch_size, atomic_number, atomic_number, 3)
edge_features = torch.randn(batch_size, atomic_number, atomic_number, 3)



In [88]:
batch_idx, rows_idx, cols_idx = torch.nonzero(atomic_adjacency_matrix, as_tuple=True)


In [89]:
batch_idx

tensor([ 0,  0,  0,  ..., 31, 31, 31])

In [91]:
batch_idx.shape

torch.Size([147791])

tensor([ 0,  0,  0,  ..., 95, 95, 95])

294912